In [ ]:
### Imviz profiling

import numpy as np
from astropy.nddata import NDData
from astropy.wcs import WCS
from jdaviz import Imviz

In [ ]:
# image size parameter (in pixels per side)
# (500, 1000, 10_000, 100_000) pixels per side
image_pixel_side = {image_pixel_side_value}

# viewport size parameter (in pixels)
# (600, 1000, 2000, 4000) pixels per side
viewport_pixel_size = {viewport_pixel_size_value}

# number of large images loaded/generated simultaneously and WCS-linked
# (1, 3, 5, 10, 25) images
n_images = {n_images_value}

# compress images generated in the python kernel, with this method before sending to the client
# ('png', 'webp', 'none')
image_compression = {image_compression_value}

# the value in megabytes to which the network should be throttled to
# (0.5, 1, 5, 10, 50, 100) megabytes
ui_network_throttling = {ui_network_throttling_value}

In [ ]:
shape = (image_pixel_side, image_pixel_side)

# one side of the pixel, in degrees
pixel_width = 0.005

# offset the centers of the fields of view
# for the three images by this fraction of
# one image's width:
center_offset = 0.2
delta_ra_offset = center_offset * pixel_width * shape[0]

wcs_list = [
    WCS({
    'CTYPE1': 'RA---TAN',
    'CUNIT1': 'deg',
    'CDELT1': pixel_width,
    'CRPIX1': shape[0] / 2,
    'CRVAL1': 180 + i * delta_ra_offset,
    'CTYPE2': 'DEC--TAN',
    'CUNIT2': 'deg',
    'CDELT2': pixel_width,
    'CRPIX2': shape[1] / 2,
    'CRVAL2': 0
    }) for i in range(n_images)
]

random_shape = (6, 6)
image_list = [np.random.uniform(size=random_shape) for i in range(n_images)]

colors = ['c', 'y', 'm', 'r', 'g', 'b', 'o']

def low_pass_filter(x):
    rfft = np.fft.rfft2(x)
    irfft = np.fft.irfft2(rfft, shape)
    return 2**16 * irfft / irfft.max()

nddata_cym = [
    NDData(low_pass_filter(image), wcs=wcs)
    for image, wcs in zip(image_list, wcs_list)
]

In [ ]:
### Initialize and show Imviz

imviz = Imviz()
imviz.app.get_viewer('imviz-0')._composite_image.compression = image_compression

label_mouseover = imviz.app.session.application._tools['g-coords-info']
viewer = imviz.default_viewer._obj

imviz.show('sidecar:split-right', height=viewport_pixel_size)

In [ ]:
### Load the images into Imviz

with imviz.batch_load():
    for nddata, data_label in zip(nddata_cym, colors):
        imviz.load_data(nddata, data_label=data_label)

compression = imviz.app.get_viewer('imviz-0')._composite_image.compression
print(f"glue-jupyter compression configuration: '{compression}'")

In [ ]:
### Use WCS to align the images, zoom to fit

orientation = imviz.plugins['Orientation']
orientation.align_by = 'WCS'

In [ ]:
### Rotate the image orientation by 180 deg counter-clockwise

# # This seems not to work, fails on call that's effectively `viewer.zoom(0)`. Bug?
# rotate the image by 180 deg (CCW):
# orientation.add_orientation(rotation_angle=180)

In [ ]:
### Assign colors to each of the layers

plot_options = imviz.plugins["Plot Options"]
for layer, color in zip(plot_options.layer.choices, colors):
    plot_options.layer = layer
    plot_options.image_color_mode = 'Color'
    plot_options.image_color = color
    plot_options.image_opacity = 0.7

In [ ]:
### Return to the default orientation

# # This seems not to work, fails on call that's effectively `viewer.zoom(0)`. Bug?
# orientation.orientation = 'Default orientation'

In [ ]:
### Blink through each layer (one visible at a time)

for i in range(len(plot_options.layer.choices)):
    viewer.blink_once()

In [ ]:
### Make all layers visible again

for layer in plot_options.layer.choices:
    plot_options.layer = layer
    plot_options.image_visible = True

In [ ]:
### Increment through colormap upper limits for each layer

for vmax in np.linspace(0, 2**16, 10):
    for layer in plot_options.layer.choices:
        plot_options.layer = layer
        plot_options.stretch_vmin = 0
        plot_options.stretch_vmax = vmax